In [1]:
import numpy as np

In [2]:
class vertex:
    def __init__(self, vertex_name):
        self.neighbourhood = []
        # ANY kind of ID. For Heidelberg, could be latitude, longitude, altitude...
        self.name = vertex_name
    def is_neighbour_of(self, vertex):
        self.neighbourhood.append(vertex)
    def print(self):
        print(self.name)        
        
class edge:
    def __init__(self, a, b, w):
        self.v1 = a
        self.v2 = b
        self.weight = w      
    def print(self):
        print(self.v1.name, "to", self.v2.name, "taking", self.weight, "km")
    
class graph:  
    def __init__(self,v1,v2,w):
        self.vertices = []
        self.dic_vertices = {}
        self.edges = []
        self.dic_edges = {}
        #adjacency matrix is composed of np.infty and weights; 
        self.weighted_adjacency_matrix = np.ones((1))*np.infty 
        
        #symmetric TSP        
        v1.is_neighbour_of(v2)
        v2.is_neighbour_of(v1)
        
        self.dic_edges[(v1.name,v2.name)]=len(self.edges)
        self.edges.append(edge(v1,v2,w))
                
        self.dic_vertices[v1.name]=len(self.vertices)
        self.vertices.append(v1)
        self.dic_vertices[v2.name]=len(self.vertices)
        self.vertices.append(v2)   
                
        a = (len(self.vertices), len(self.vertices))
        
        temp_matrix = np.ones(a)*np.infty
        #symmetric TSP                      
        temp_matrix[self.dic_vertices[v1.name], self.dic_vertices[v2.name]] = w
        temp_matrix[self.dic_vertices[v2.name], self.dic_vertices[v1.name]] = w

        temp_matrix[:-1,:-1] = self.weighted_adjacency_matrix
        self.weighted_adjacency_matrix =np.copy(temp_matrix)
        
        
    def attach_vertex(self, new_vertex, existing_vertex, weight):        

        new_edge = edge(new_vertex, existing_vertex, weight)
        
        self.dic_edges[(new_edge.v1.name,new_edge.v2.name)] = len(self.edges) 
        self.edges.append(new_edge)
        
        self.dic_vertices[new_vertex.name]=len(self.vertices)
        self.vertices.append(new_vertex)
        
        
        #append a column and a row to the symmetric adjacency matrix, where the (new_vertex, existing_vertex) entries
        #are updated with weight, everything else set to zero
        
        #dictionary of names to indices makes the substitution straightforward
             
        # new matrix with an extra row and column
        a = (len(self.vertices), len(self.vertices))
        temp_matrix = np.ones(a)*np.infty
        # copy the weight to the appropriate place and vice versa (symmetric TSP)
        temp_matrix[self.dic_vertices[new_vertex.name],self.dic_vertices[existing_vertex.name]] = weight
        temp_matrix[self.dic_vertices[existing_vertex.name],self.dic_vertices[new_vertex.name]] = weight
        #and copy the old matrix entries onto the new matriz entries
        temp_matrix[:-1,:-1] = self.weighted_adjacency_matrix
        self.weighted_adjacency_matrix =np.copy(temp_matrix)
        
        #symmetric TSP
        self.vertices[self.dic_vertices[new_vertex.name]].is_neighbour_of(self.vertices[self.dic_vertices[existing_vertex.name]])
        self.vertices[self.dic_vertices[existing_vertex.name]].is_neighbour_of(self.vertices[self.dic_vertices[new_vertex.name]])
             
    def attach_edge(self, v1, v2, weight):
        
        new_edge = edge(v1, v2, weight)
        self.dic_edges[(new_edge.v1.name,new_edge.v2.name)] = len(self.edges) 

        self.edges.append(new_edge)
        #symmetric TSP
        self.weighted_adjacency_matrix[self.dic_vertices[v1.name],self.dic_vertices[v2.name]] = weight
        self.weighted_adjacency_matrix[self.dic_vertices[v2.name],self.dic_vertices[v1.name]] = weight

        self.vertices[self.dic_vertices[v1.name]].is_neighbour_of(self.vertices[self.dic_vertices[v2.name]])
        self.vertices[self.dic_vertices[v2.name]].is_neighbour_of(self.vertices[self.dic_vertices[v1.name]])
        
    def remove_edge(self, v1, v2):
        #symmetric TSP
        # remove edge and dictionary reference from the graph
        try:
            index = self.dic_edges[(v1.name,v2.name)]
        except:
            index = self.dic_edges[(v2.name,v1.name)]
        del self.edges[index]
        
        try:
            del self.dic_edges[(v1.name,v2.name)]
        except:
            del self.dic_edges[(v2.name,v1.name)]
            
        # update adjacency matrix entries to infinity
        self.weighted_adjacency_matrix[self.dic_vertices[v1.name],self.dic_vertices[v2.name]] = np.infty
        self.weighted_adjacency_matrix[self.dic_vertices[v2.name],self.dic_vertices[v1.name]] = np.infty
        # remove vertices from each other's neighbourhood
        self.vertices[self.dic_vertices[v1.name]].neighbourhood.remove(self.vertices[self.dic_vertices[v2.name]])
        self.vertices[self.dic_vertices[v2.name]].neighbourhood.remove(self.vertices[self.dic_vertices[v1.name]])

        
    def update_edge(self, v1, v2, new_weight):
        #symmetric TSP
        try:
            index = self.dic_edges[(v1.name,v2.name)]
        except:
            index = self.dic_edges[(v2.name,v1.name)]
            
        self.edges[index].weight = new_weight   
        self.weighted_adjacency_matrix[self.dic_vertices[v1.name],self.dic_vertices[v2.name]] = new_weight
        self.weighted_adjacency_matrix[self.dic_vertices[v2.name],self.dic_vertices[v1.name]] = new_weight
        
    def get_edge(self,v1,v2):
        """returns a list with the index and weight of the edge between v1 and v2 (symmetric)"""
        try:
            index = self.dic_edges[(v1.name,v2.name)]
        except:
            index = self.dic_edges[(v2.name,v1.name)]
        return [index, self.edges[index].weight]
                    
    def attach_vertex_fully_connected(self, new_vertex, weight_function):
        """used to build fully connected graphs (Heidelberg)"""
        vertex_list = self.vertices.copy()
        self.attach_vertex(new_vertex, vertex_list[0], weight_function(new_vertex, vertex_list[0]))
        
        for existing_vertex in vertex_list[1:]:
            self.attach_edge(new_vertex, existing_vertex, weight_function(new_vertex, existing_vertex))
            
    def print(self):
        """prints general info"""
        names = [a.name for a in self.vertices]
        print(len(self.vertices), "VERTICES:",', '.join(names),"\n")
        weights = [str(a.weight)+" km" for a in self.edges]
        print(len(self.edges), "EDGES:",', '.join(weights))
        print("\nADJACENCY MATRIX:\n",self.weighted_adjacency_matrix)

In [3]:
def get_node_coordinates_2D(filename):
    """gets node coordinates in 2D. Should work with any distance"""
    node_coordinates = []

    with open(filename) as input_data:
        interesting_lines = []
    
        for line in input_data:
            if line.strip() == 'NODE_COORD_SECTION':  #read from this point
                break
            
        for line in input_data:
            if line.strip() == 'EOF': #until this point
                break
            else:
                interesting_lines.append(list(map(float,line.split()))) #some files have ints in floating point notation

    input_data.close()
    node_coordinates = np.array(interesting_lines)[:,[1,2]]
    return node_coordinates

In [4]:
def nearest_int(x):
    if x-int(x)<int(x+1)-x:
        return int(x)
    else:
        return int(x+1)

In [5]:
def nearest_int_euclidean_distance_2D(v1,v2):
    """between vertices, rounded to the nearest integer,
    as required in the TSPLIB docs"""
    xd = v1.name[0] - v2.name[0]
    yd = v1.name[1] - v2.name[1]
    return nearest_int(np.sqrt(xd*xd + yd*yd))

In [6]:
def heidelberg_2D(filename):
    """parses 2D distance TSP type .tsp file from Heidelberg and returns the graph"""
    nodes = get_node_coordinates_2D(filename)
    v1 = vertex(tuple(nodes[0]))
    v2 = vertex(tuple(nodes[1]))
    heidelberg_graph = graph(v1,v2, nearest_int_euclidean_distance_2D(v1,v2))
    for node in nodes[2:]:
        heidelberg_graph.attach_vertex_fully_connected(vertex(tuple(node)), nearest_int_euclidean_distance_2D)
    return heidelberg_graph

#### Creating a graph with two vertices named "Cairo" and "Paris", at a distance of 5000km:

In [7]:
first_graph = graph(vertex("Cairo"),vertex("Paris"), 5000)
first_graph.print()

2 VERTICES: Cairo, Paris 

1 EDGES: 5000 km

ADJACENCY MATRIX:
 [[  inf 5000.]
 [5000.   inf]]


#### The distance between Paris and Cairo is actually 5418.9 km. Note that the ``update_edge()`` method is symmetric as a function of the vertices.

In [8]:
first_graph.update_edge(vertex("Paris"), vertex("Cairo"),5418.9)
first_graph.print()

2 VERTICES: Cairo, Paris 

1 EDGES: 5418.9 km

ADJACENCY MATRIX:
 [[   inf 5418.9]
 [5418.9    inf]]


#### We can remove the edge between the two cities:

In [9]:
first_graph.remove_edge(vertex("Cairo"),vertex("Paris"))
first_graph.print()

2 VERTICES: Cairo, Paris 

0 EDGES: 

ADJACENCY MATRIX:
 [[inf inf]
 [inf inf]]


#### Note that the neighbourhood of the two vertices is now empty:

In [10]:
first_graph.vertices[0].neighbourhood, first_graph.vertices[1].neighbourhood

([], [])

#### We can attach the edge back in place:

In [11]:
first_graph.attach_edge(vertex("Cairo"), vertex("Paris"), 5418.9)
first_graph.print()

2 VERTICES: Cairo, Paris 

1 EDGES: 5418.9 km

ADJACENCY MATRIX:
 [[   inf 5418.9]
 [5418.9    inf]]


#### And the neighbourhoods are updated as well

In [12]:
first_graph.vertices[0].neighbourhood, first_graph.vertices[1].neighbourhood

([<__main__.vertex at 0x7fa94a44b7b8>], [<__main__.vertex at 0x7fa94a44bac8>])

#### Attach a new city to a city in the graph updates the adjacency matrix as well:

In [13]:
first_graph.attach_vertex(vertex("Saint Etienne"), vertex("Cairo"),6000)
first_graph.print()

3 VERTICES: Cairo, Paris, Saint Etienne 

2 EDGES: 5418.9 km, 6000 km

ADJACENCY MATRIX:
 [[   inf 5418.9 6000. ]
 [5418.9    inf    inf]
 [6000.     inf    inf]]


#### Note that it also updates the neighbourhood of all vertices

In [14]:
first_graph.vertices[0].neighbourhood, first_graph.vertices[1].neighbourhood, first_graph.vertices[2].neighbourhood

([<__main__.vertex at 0x7fa94a44b7b8>, <__main__.vertex at 0x7fa94a3ff550>],
 [<__main__.vertex at 0x7fa94a44bac8>])

#### Parse the file by passing the file name to the ``heidelberg_2D`` function

In [15]:
file = './data/a280.tsp'
heidelberg_graph = heidelberg_2D(file)

#### ``heidelberg_graph.weighted_adjacency_matrix`` stores the graph's adjacency matrix as a numpy array:

In [16]:
heidelberg_graph.weighted_adjacency_matrix

array([[inf, 20., 24., ..., 43., 34., 18.],
       [20., inf, 18., ..., 36., 28.,  9.],
       [24., 18., inf, ..., 20., 11., 10.],
       ...,
       [43., 36., 20., ..., inf,  9., 29.],
       [34., 28., 11., ...,  9., inf, 20.],
       [18.,  9., 10., ..., 29., 20., inf]])

#### ``heidelberg_graph.vertices`` stores all the vertices and ``heidelberg_graph.edges`` all the edges:

In [17]:
heidelberg_graph.vertices[:5], heidelberg_graph.edges[:5]

([<__main__.vertex at 0x7fa960082550>,
  <__main__.edge at 0x7fa94a3ff128>])

#### Use the ``graph.get_edge()`` method to get the index and weight of the edge between two vertices. Note that it matches the entry in the adjacency matrix!

In [18]:
a = heidelberg_graph.vertices[0]
b = heidelberg_graph.vertices[21]
heidelberg_graph.get_edge(a,b), heidelberg_graph.weighted_adjacency_matrix[0,21]

([210, 141], 141.0)